# Simple System Manager - Parameter Store

The simple system manager has a parameter store to allow parameters to be stored and accessed. It can be secured using standard AWS security, can encrypt credentials, and so on.

In [ ]:
import boto3

ssm_client = boto3.client('ssm')

In [ ]:
for i in range(1,26):
    key = 'sample.' + str(i)
    val = 'Value' + str(i)
    ssm_client.put_parameter(
        Name=key,
        Value=val,
        Type='String',
        Overwrite=True
    )
    
    

In [ ]:
# Store other some values
ssm_client.put_parameter(
    Name='prod.p1',
    Description='Parameter p1 in sample namespace',
    Value='p1 value prod',
    Type='String',
    Overwrite=True
)

ssm_client.put_parameter(
    Name='dev.p1',
    Description='Parameter p1 in sample namespace',
    Value='p1 value dev',
    Type='String',
    Overwrite=True
)

In [ ]:
all_done = 'that rug really tied the room together'
response = ssm_client.describe_parameters()
print response

for i in range (1,100):
    nextToken = response.get('NextToken',all_done)
    if nextToken == all_done:
        break
        
    print '=====> Call again'
    response = ssm_client.describe_parameters(
        NextToken=nextToken
    )
    print response

In [ ]:
# Retrieve a value

response = ssm_client.get_parameters(
    Names=[
        'sample.p1',
    ]
)

params = response['Parameters']
print 'Value of sample.p1: \'{}\''.format(next((x['Value'] for x in params if x['Name'] == 'sample.p1'), None))
print 'Value of foo: {}'.format(next((x['Value'] for x in params if x['Name'] == 'foo'), None))

## Encrypted Values

### Create a Key

In [ ]:
client = boto3.client('kms')
response = client.create_key(
    Description='sample-key',
)

print response

In [ ]:
key_id = response['KeyMetadata']['KeyId']
print key_id

In [ ]:
response = ssm_client.put_parameter(
    Name='sample.my_secret',
    Value='seriously do not tell',
    Type='SecureString',
    KeyId=key_id
)

In [ ]:
response = ssm_client.get_parameters(
    Names=[
        'sample.my_secret',
    ]
)

params = response['Parameters']
print params

In [ ]:
response = ssm_client.get_parameters(
    Names=[
        'sample.my_secret',
    ],
    WithDecryption=True
)
print response['Parameters']